# Welcome to Full Stack Machine Learning's Week 4 Project!

In the final week, you will return to the workflow you built last week on the [taxi dataset](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page). 

## Task 1: Deploy the champion
Use what you have learned in the last two weeks to make necessary modifications and to deploy your latest version of the `TaxiFarePrediction` flow to Argo. Use `--branch champion` to denote this deployment as the champion model.

In [1]:
%%writefile ../flows/cloud/taxi_champion.py
from metaflow import FlowSpec, step, card, conda_base, current, Parameter, Flow, trigger, project, S3
from metaflow.cards import Markdown, Table, Image, Artifact

# URL = "https://outerbounds-datasets.s3.us-west-2.amazonaws.com/taxi/latest.parquet"
URL = 's3://outerbounds-datasets/taxi/latest.parquet'
DATETIME_FORMAT = '%Y-%m-%d %H:%M:%S'

# @trigger(events=['s3'])
@conda_base(libraries={'pandas': '1.4.2', 'pyarrow': '11.0.0', 'numpy': '1.21.2', 'scikit-learn': '1.1.2', 'py-xgboost': '1.7.4'})
@project(name="taxi_model")
class TaxiModel(FlowSpec):

    data_url = Parameter("data_url", default=URL)

    def transform_features(self, df):

        # TODO: 
            # Try to complete tasks 2 and 3 with this function doing nothing like it currently is.
            # Understand what is happening.
            # Revisit task 1 and think about what might go in this function.

        obviously_bad_data_filters = [

            df.fare_amount > 0,         # fare_amount in US Dollars
            df.trip_distance <= 100,    # trip_distance in miles
            df.trip_distance > 0,

            # TODO: add some logic to filter out what you decide is bad data!
            # TIP: Don't spend too much time on this step for this project though, it practice it is a never-ending process.
            df.extra >= 0,
            df.mta_tax >= 0,
            df.tip_amount >= 0,
            df.total_amount > 0,
            df.airport_fee >= 0,

        ]

        for f in obviously_bad_data_filters:
            df = df[f]
        df = df.dropna()
        return df

    @step
    def start(self):

        import pandas as pd
        from sklearn.model_selection import train_test_split
        with S3() as s3:
            obj = s3.get(URL)
            df = pd.read_parquet(obj.path)

        self.df = self.transform_features(df)
        
        # self.df = self.transform_features(pd.read_parquet(self.data_url))

        # NOTE: we are split into training and validation set in the validation step which uses cross_val_score.
        # This is a simple/naive way to do this, and is meant to keep this example simple, to focus learning on deploying Metaflow flows.
        # In practice, you want split time series data in more sophisticated ways and run backtests. 
        self.X = self.df["trip_distance"].values.reshape(-1, 1)
        self.y = self.df["total_amount"].values
        self.next(self.linear_model)

    @step
    def linear_model(self):
        "Fit a single variable, linear model to the data."
        from sklearn.linear_model import LinearRegression
        # TODO: Play around with the model if you are feeling it.
        self.model = LinearRegression()

        self.next(self.validate)

    def gather_sibling_flow_run_results(self):

        # storage to populate and feed to a Table in a Metaflow card
        rows = []

        # loop through runs of this flow 
        for run in Flow(self.__class__.__name__):
            if run.id != current.run_id:
                if run.successful:
                    icon = "✅" 
                    msg = "OK"
                    score = str(run.data.scores.mean())
                else:
                    icon = "❌"
                    msg = "Error"
                    score = "NA"
                    for step in run:
                        for task in step:
                            if not task.successful:
                                msg = task.stderr
                row = [Markdown(icon), Artifact(run.id), Artifact(run.created_at.strftime(DATETIME_FORMAT)), Artifact(score), Markdown(msg)]
                rows.append(row)
            else:
                rows.append([Markdown("✅"), Artifact(run.id), Artifact(run.created_at.strftime(DATETIME_FORMAT)), Artifact(str(self.scores.mean())), Markdown("This run...")])
        return rows
                
    
    @card(type="corise")
    @step
    def validate(self):
        from sklearn.model_selection import cross_val_score
        self.scores = cross_val_score(self.model, self.X, self.y, cv=5)
        current.card.append(Markdown("# Taxi Fare Prediction Results"))
        current.card.append(Table(self.gather_sibling_flow_run_results(), headers=["Pass/fail", "Run ID", "Created At", "R^2 score", "Stderr"]))
        self.next(self.end)

    @step
    def end(self):
        print("Success!")


if __name__ == "__main__":
    TaxiModel()

Overwriting ../flows/cloud/taxi_champion.py


In [2]:
# test local
! python ../flows/cloud/taxi_champion.py --environment=conda run

Metaflow 2.9.1+ob(v1) executing TaxiModel for user:sandbox
Project: taxi_model, Branch: user.sandbox
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Bootstrapping conda environment...(this could take a few minutes)
2023-05-21 14:14:43.438 Workflow starting (run-id 94), see it in the UI at https://ui-pw-1752302777.outerbounds.dev/TaxiModel/94
2023-05-21 14:14:44.540 [94/start/301 (pid 3526)] Task is starting.
2023-05-21 14:14:51.083 [94/start/301 (pid 3526)] Task finished successfully.
2023-05-21 14:14:51.951 [94/linear_model/302 (pid 3613)] Task is starting.
2023-05-21 14:14:54.404 [94/linear_model/302 (pid 3613)] Task finished successfully.
2023-05-21 14:14:55.310 [94/validate/303 (pid 3660)] Task is starting.
2023-05-21 14:15:01.739 [94/validate/303 (pid 3660)] Task finished successfully.
2023-05-21 14:15:02.890 [94/end/304 (pid 3721)] Task is starting.
2023-05-21 14:15:03.997 [94/end/304 (pid 3721)] Success!
2023-05-21 14:15:04.932 [94/end/30

In [3]:
! python ../flows/cloud/taxi_champion.py --environment=conda --production --branch champion --production argo-workflows create

Metaflow 2.9.1+ob(v1) executing TaxiModel for user:sandbox
Project: taxi_model, Branch: prod.champion
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Deploying taximodel.prod.champion.taximodel to Argo Workflows...

The namespace of this production flow is
    production:mfprj-obh3cwsqu6t3gnrz-0-erpv
To analyze results of this production flow add this line in your notebooks:
    namespace("production:mfprj-obh3cwsqu6t3gnrz-0-erpv")
If you want to authorize other people to deploy new versions of this flow to Argo Workflows, they need to call
    argo-workflows create --authorize mfprj-obh3cwsqu6t3gnrz-0-erpv
when deploying this flow to Argo Workflows for the first time.
See "Organizing Results" at https://docs.metaflow.org/ for more information about production tokens.

Bootstrapping conda environment...(this could take a few minutes)
Workflow taximodel.prod.champion.taximodel for flow TaxiModel pushed to Argo Workflows successfully.

Note that t

In [4]:
! python ../flows/cloud/taxi_champion.py --environment=conda --production --branch champion --production argo-workflows trigger


Metaflow 2.9.1+ob(v1) executing TaxiModel for user:sandbox
Project: taxi_model, Branch: prod.champion
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Workflow taximodel.prod.champion.taximodel triggered on Argo Workflows (run-id argo-taximodel.prod.champion.taximodel-cwr7b).
See the run in the UI at https://ui-pw-1752302777.outerbounds.dev/TaxiModel/argo-taximodel.prod.champion.taximodel-cwr7b


## Task 2: Build the challenger
Develop a second model, by using the same `TaxiFarePrediction` architecture. Then, deploy the flow to Argo as the `--branch challenger`. 
<br>
<br>
Hint: Modify the `linear_model` step. 
<br>
Bonus: Write a paragraph summary of how you developed the second model and tested it before deploying the challenger flow. Let us know in Slack what you found challenging about the task? 

In [5]:
%%writefile ../flows/cloud/taxi_challenger.py
from metaflow import FlowSpec, step, card, conda_base, current, Parameter, Flow, trigger, project, S3, retry
from metaflow.cards import Markdown, Table, Image, Artifact

# URL = "https://outerbounds-datasets.s3.us-west-2.amazonaws.com/taxi/latest.parquet"
URL = 's3://outerbounds-datasets/taxi/latest.parquet'
DATETIME_FORMAT = '%Y-%m-%d %H:%M:%S'

# @trigger(events=['s3'])
@conda_base(libraries={'pandas': '1.4.2', 'pyarrow': '11.0.0', 'numpy': '1.21.2', 'scikit-learn': '1.1.2', 'py-xgboost': '1.7.4'})
@project(name="taxi_model")
class TaxiModel(FlowSpec):

    data_url = Parameter("data_url", default=URL)

    def transform_features(self, df):

        # TODO: 
            # Try to complete tasks 2 and 3 with this function doing nothing like it currently is.
            # Understand what is happening.
            # Revisit task 1 and think about what might go in this function.

        obviously_bad_data_filters = [

            df.fare_amount > 0,         # fare_amount in US Dollars
            df.trip_distance <= 100,    # trip_distance in miles
            df.trip_distance > 0,

            # TODO: add some logic to filter out what you decide is bad data!
            # TIP: Don't spend too much time on this step for this project though, it practice it is a never-ending process.
            df.extra >= 0,
            df.mta_tax >= 0,
            df.tip_amount >= 0,
            df.total_amount > 0,
            df.airport_fee >= 0,

        ]

        for f in obviously_bad_data_filters:
            df = df[f]
        df = df.dropna()
        return df

    @retry(times=2)  # maybe a read error
    @step
    def start(self):

        import pandas as pd
        from sklearn.model_selection import train_test_split

        with S3() as s3:
            obj = s3.get(URL)
            df = pd.read_parquet(obj.path)

        self.df = self.transform_features(df)
        #self.df = self.transform_features(pd.read_parquet(self.data_url))

        # NOTE: we are split into training and validation set in the validation step which uses cross_val_score.
        # This is a simple/naive way to do this, and is meant to keep this example simple, to focus learning on deploying Metaflow flows.
        # In practice, you want split time series data in more sophisticated ways and run backtests. 
        self.X = self.df["trip_distance"].values.reshape(-1, 1)
        self.y = self.df["total_amount"].values
        self.next(self.linear_model)

    @step
    def linear_model(self):
        "Fit a single variable, linear model to the data."
        # from sklearn.linear_model import LinearRegression
        from xgboost import XGBRegressor
        # TODO: Play around with the model if you are feeling it.
        # self.model = LinearRegression()
        self.model = XGBRegressor()

        self.next(self.validate)

    def gather_sibling_flow_run_results(self):

        # storage to populate and feed to a Table in a Metaflow card
        rows = []

        # loop through runs of this flow 
        for run in Flow(self.__class__.__name__):
            if run.id != current.run_id:
                if run.successful:
                    icon = "✅" 
                    msg = "OK"
                    score = str(run.data.scores.mean())
                else:
                    icon = "❌"
                    msg = "Error"
                    score = "NA"
                    for step in run:
                        for task in step:
                            if not task.successful:
                                msg = task.stderr
                row = [Markdown(icon), Artifact(run.id), Artifact(run.created_at.strftime(DATETIME_FORMAT)), Artifact(score), Markdown(msg)]
                rows.append(row)
            else:
                rows.append([Markdown("✅"), Artifact(run.id), Artifact(run.created_at.strftime(DATETIME_FORMAT)), Artifact(str(self.scores.mean())), Markdown("This run...")])
        return rows
                
    
    @card(type="corise")
    @step
    def validate(self):
        from sklearn.model_selection import cross_val_score
        self.scores = cross_val_score(self.model, self.X, self.y, cv=5)
        current.card.append(Markdown("# Taxi Fare Prediction Results"))
        current.card.append(Table(self.gather_sibling_flow_run_results(), headers=["Pass/fail", "Run ID", "Created At", "R^2 score", "Stderr"]))
        self.next(self.end)

    @step
    def end(self):
        print("Success!")


if __name__ == "__main__":
    TaxiModel()

Overwriting ../flows/cloud/taxi_challenger.py


In [6]:
# test local
# ! python ../flows/cloud/taxi_challenger.py --environment=conda run

In [7]:
! python ../flows/cloud/taxi_challenger.py --environment=conda --production --branch challenger --production argo-workflows create

Metaflow 2.9.1+ob(v1) executing TaxiModel for user:sandbox
Project: taxi_model, Branch: prod.challenger
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Deploying taximodel.prod.challenger.taximodel to Argo Workflows...

The namespace of this production flow is
    production:mfprj-aavz62ho7mcnmcl5-0-sxgl
To analyze results of this production flow add this line in your notebooks:
    namespace("production:mfprj-aavz62ho7mcnmcl5-0-sxgl")
If you want to authorize other people to deploy new versions of this flow to Argo Workflows, they need to call
    argo-workflows create --authorize mfprj-aavz62ho7mcnmcl5-0-sxgl
when deploying this flow to Argo Workflows for the first time.
See "Organizing Results" at https://docs.metaflow.org/ for more information about production tokens.

Bootstrapping conda environment...(this could take a few minutes)
Workflow taximodel.prod.challenger.taximodel for flow TaxiModel pushed to Argo Workflows successfully.

Note 

In [8]:
! python ../flows/cloud/taxi_challenger.py --environment=conda --production --branch challenger --production argo-workflows trigger


Metaflow 2.9.1+ob(v1) executing TaxiModel for user:sandbox
Project: taxi_model, Branch: prod.challenger
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Workflow taximodel.prod.challenger.taximodel triggered on Argo Workflows (run-id argo-taximodel.prod.challenger.taximodel-btf9r).
See the run in the UI at https://ui-pw-1752302777.outerbounds.dev/TaxiModel/argo-taximodel.prod.challenger.taximodel-btf9r


## Task 3: Analyze the results
Return to this notebook, and read in the results of the challenger and champion flow using the Metaflow Client API.
<br><br>

#### Questions
- Does your model perform better on the metrics you selected? 
- Think about your day job, how would you go about assessing whether to roll forward the production "champion" to your new model? 
    - What gives you confidence one model is better than another?
    - What kinds of information do you need to monitor to get buy-in from stakeholders that model A is preferable to model B?  

In [15]:
from metaflow import Flow, namespace
import numpy as np
# these values are unique to your deployment!
CHAMPION_MODEL_NAMESPACE = "production:mfprj-obh3cwsqu6t3gnrz-0-erpv" # "production:mfprj-xsfdb3gtsiboqyrd-0-vqsy"
CHALLENGER_MODEL_NAMESPACE = "production:mfprj-aavz62ho7mcnmcl5-0-sxgl" # "production:mfprj-cfyzlfzievjlmsf4-0-tbgz"


best_score = -1; winner = None; winner_namespace = None
for n, name in zip([CHAMPION_MODEL_NAMESPACE, CHALLENGER_MODEL_NAMESPACE], ['champion', 'challenger']):
    namespace(n)
    run = Flow('TaxiModel').latest_successful_run
    acc_score = np.mean(run.data.scores)
    print(acc_score)
    if acc_score > best_score:
        best_score = acc_score
        winner_namespace = n
        winner = name
print(f' winner: {winner} with acc: {best_score}')

0.8960377585295058
0.9044956583567151
 winner: challenger with acc: 0.9044956583567151


## CONGRATULATIONS! 🎉✨🍾
If you made it this far, you have completed the Full Stack Machine Learning Corise course. 
We are so glad that you chose to learn with us, and hope to see you again in future courses. Stay tuned for more content and come join us in [Slack](http://slack.outerbounds.co/) to keep learning about Metaflow!